In [1]:
import numpy as np
import tqdm
embedding_path='raw_data/rotten_tomatoes/embeddings/glove.6B.200d.txt'
lines = []
with open(embedding_path, encoding='utf8') as file:
    lines = file.readlines()
    file.close()
embedding_tensor = []
token2id = {}
token2id['<unk>'] = 0
token2id['<pad>'] = 1
for id, l in tqdm.tqdm(enumerate(lines)):
    word, emb = l.split()[0], l.split()[1:]
    vector = [float(x) for x in emb ]
    if id == 0:
        embedding_tensor.append( np.zeros( len(vector) ) )
        embedding_tensor.append( np.zeros( len(vector) ) )
    embedding_tensor.append(vector)
    token2id[word] = id+2
embedding_tensor = np.array(embedding_tensor, dtype=np.float32)

400000it [01:43, 3858.58it/s] 


In [2]:
import pandas as pd
from utils.embedding import get_indices_tensor
from utils.contractions import CONTRACTIONS_DICT
import re
def clean_punctuation(text):
    text = re.sub("""([!"#$%&'()*+, -./:;<=>?@[\]^_`{|}~])""", r' \1 ', text)
    text = re.sub('\s{2,}', ' ', text)
    return text
def expand_contractions(text):
    contractions_re=re.compile('(%s)' % '|'.join(CONTRACTIONS_DICT.keys()))
    def replace(match):
        return CONTRACTIONS_DICT[match.group(0)]
    return contractions_re.sub(replace, text)
def clean_spaces(text):
    text = " ".join(text.split())
    return text
root_path = 'raw_data/rotten_tomatoes/data/audience_reviews.csv'
df = pd.read_csv(root_path).dropna()
df['text']= df['Review'].apply(lambda x: x.lower())
df['text']=df['text'].apply(lambda x:expand_contractions(x))
df['text'] = df['text'].apply(lambda x:clean_spaces(x))
df['text']= df['text'].apply(lambda x:clean_punctuation(x))
df['text_list']= df['text'].apply(lambda x: x.split())
df['y'] = df['Rating'].apply(lambda x: x/5)
df['id'] = range(0, len(df))
df['x'] = df['text_list'].apply(lambda x: get_indices_tensor(x, token2id, 250))

In [3]:
df.head()

,Show,Rating,Review,text,text_list,y,id,x
0,Loot,2.5,"I love the stars, but the show is riddled with...","i love the stars , but the show is riddled wit...","[i, love, the, stars, ,, but, the, show, is, r...",0.5,0,"[[tensor(43), tensor(837), tensor(2), tensor(1..."
1,Loot,4.5,Great show to watch. It's something that's eas...,great show to watch . it is something that is ...,"[great, show, to, watch, ., it, is, something,...",0.9,1,"[[tensor(355), tensor(275), tensor(6), tensor(..."
2,Loot,0.5,Another Apple tanker crashes into the reef wit...,another apple tanker crashes into the reef wit...,"[another, apple, tanker, crashes, into, the, r...",0.1,2,"[[tensor(172), tensor(3294), tensor(8406), ten..."
3,Loot,4.5,"This is such a fun, endearing show. It has a ...","this is such a fun , endearing show . it has a...","[this, is, such, a, fun, ,, endearing, show, ....",0.9,3,"[[tensor(39), tensor(16), tensor(127), tensor(..."
4,Loot,0.5,Terrible show - didn't laugh once. Acting is s...,terrible show - did not laugh once . acting is...,"[terrible, show, -, did, not, laugh, once, ., ...",0.1,4,"[[tensor(5512), tensor(275), tensor(13), tenso..."


In [4]:
data = df[['text', 'x', 'y', 'id']].to_dict('records')

In [5]:
data[0:2]

[{'text': 'i love the stars , but the show is riddled with cliche characters and comedy beats as well as relentlessly predictable story lines . these feel like characters who are shorthand for characters we have already seen , which sometimes is the curse of a new tv show , especially sitcoms . so i guess we will see ? ',
  'x': tensor([[   43,   837,     2,  1571,     3,    36,     2,   275,    16, 16472,
              19, 25383,  2155,     7,  2843,  6875,    21,   145,    21, 19340,
           12672,   525,  1332,     4,   160,  1000,   119,  2155,    40,    34,
           27316,    12,  2155,    55,    35,   413,   543,     3,    44,  1073,
              16,     2, 12625,     5,     9,    52,   818,   275,     3,   860,
           25166,     4,   102,    43,  5022,    55,    45,   255,   190,     1,
               1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
               1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
               1,     1